<a href="https://colab.research.google.com/github/marco-siino/fake_news_spreaders_detection/blob/main/FNS_RoBERTa_MSiino_ModelNB_noHashtag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset. 
RoBERTa Model, Training and Testing Notebook.
Code by M. Siino. 

From the paper: "Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset." by M.Siino et al.

## Importing modules.

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import torch
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from io import open
from pathlib import Path
from simpletransformers.classification import ClassificationModel, ClassificationArgs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing DS and extract in current working directory.

In [ ]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-training-2020-02-23.zip"
urlTestSet="https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-test-2020-02-23.zip"

training_set = tf.keras.utils.get_file("pan20-author-profiling-training-2020-02-23.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan20-author-profiling-test-2020-02-23.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan20-author-profiling-training-2020-02-23')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan20-author-profiling-test-2020-02-23')

print(training_set)
print(training_set_dir)

!ls -A

./pan20-author-profiling-training-2020-02-23.zip
./pan20-author-profiling-training-2020-02-23
.config						runs
__MACOSX					sample_data
outputs						test_dir_en
pan20-author-profiling-test-2020-02-23		test_dir_es
pan20-author-profiling-test-2020-02-23.zip	train_dir_en
pan20-author-profiling-training-2020-02-23	train_dir_es
pan20-author-profiling-training-2020-02-23.zip


## Build folders hierarchy to use Keras folders preprocessing function.

In [ ]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')
if not os.path.exists('train_dir_es'):
    os.makedirs('train_dir_es')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_es/0'):
    os.makedirs('train_dir_es/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')
if not os.path.exists('train_dir_es/1'):
    os.makedirs('train_dir_es/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')
if not os.path.exists('test_dir_es'):
    os.makedirs('test_dir_es')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_es/0'):
    os.makedirs('test_dir_es/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')
if not os.path.exists('test_dir_es/1'):
    os.makedirs('test_dir_es/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

.config						runs
__MACOSX					sample_data
outputs						test_dir_en
pan20-author-profiling-test-2020-02-23		test_dir_es
pan20-author-profiling-test-2020-02-23.zip	train_dir_en
pan20-author-profiling-training-2020-02-23	train_dir_es
pan20-author-profiling-training-2020-02-23.zip


## Set language and directory paths.


In [ ]:
# Set en and es train_dir and test_dir paths.
language='es'

truth_file_training_dir_es=training_set_dir+'/'+language+'/'
truth_file_training_path_es = truth_file_training_dir_es+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_es = truth_file_test_dir+'/'+language+'.txt'


language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_path_en = truth_file_test_dir+'/'+language+'.txt'

## Read truth.txt to organize training dataset folders.



In [ ]:
# Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

language='es'
# Organize ES folders.
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_es+fNameXml):
      os.rename(truth_file_training_dir_es+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Read truth.txt to organize test dataset folders.

In [ ]:
#Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

#Organize EN folders.
language='es'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Function to pre-process source text.

In [ ]:
def custom_standardization(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_es_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="es">', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_author_lang_es_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  output_data_noHashtag = tf.strings.regex_replace(output_data,'#HASHTAG#', '')
  return output_data_noHashtag

## Building the dataset.

In [ ]:
batch_size=1

# Build the dataset for Spanish.
language='es'

raw_train_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


# Build the dataset for Spanish.
language='en'

raw_train_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 300 files belonging to 2 classes.
Found 200 files belonging to 2 classes.


## Convert DSs to Pandas Dataframe

In [ ]:
# Convert English dataset.
train_df_en = [] # will contain text and label
for element in raw_train_ds_en:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  train_df_en.append({
      'text':text,
      'label':label
  })
train_df_en = pd.DataFrame(train_df_en)
test_df_en = [] # will contain text and label
for element in raw_test_ds_en:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  test_df_en.append({
      'text':text,
      'label':label
  })
test_df_en = pd.DataFrame(test_df_en)

# Convert Spanish dataset.
train_df_es = [] # will contain text and label
for element in raw_train_ds_es:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  train_df_es.append({
      'text':text,
      'label':label
  })
train_df_es = pd.DataFrame(train_df_es)
test_df_es = [] # will contain text and label
for element in raw_test_ds_es:
  authorDocument=element[0]
  label=int(element[1].numpy())
  #print(authorDocument[0])
  text = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  test_df_es.append({
      'text':text,
      'label':label
  })
test_df_es = pd.DataFrame(test_df_es)

## Print some RAW and preprocessed samples (No need to execute)

In [ ]:
for idx, element in enumerate(raw_train_ds_es):
  if idx>1: break
  authorDocument=element[0]
  label=element[1]
  temp = custom_standardization(authorDocument[0].numpy()).numpy().decode('UTF-8')
  print("Not-Preprocessed samples: \n",authorDocument)
  print("Preprocessed samples: \n",temp)

Not-Preprocessed samples: 
 tf.Tensor([b'<author lang="es">\n\t<documents>\n\t\t<document><![CDATA[DESCARGAR \xe2\x80\x93 Jacool el Fenomeno \xe2\x80\x93 Me la va dar (2015) #URL# Unete #USER# #URL#]]></document>\n\t\t<document><![CDATA[VIDEO \xe2\x80\x93 Mozart la Para \xe2\x80\x93 Llegan Lo Montro Men en Lucky 7, Newark! #URL# Unete #USER#]]></document>\n\t\t<document><![CDATA[Le Quitan La Vida A Un Dominicano En Un Parque Del Bronx #URL# #URL#]]></document>\n\t\t<document><![CDATA[Historia del Joven que durante el d\xc3\xada es un millonario y de noche vuelve a la\xe2\x80\xa6 #URL# Unete #USER# #URL#]]></document>\n\t\t<document><![CDATA[VIDEO \xe2\x80\x93 El publico le grita Martha heredia a vakero en pleno concierto y esta fue la respuesta. #URL# Unete #USER#]]></document>\n\t\t<document><![CDATA[VIDEO \xe2\x80\x93 Mozart La Para y Shelow Shaq Juntos en Concierto en Los Estados Unidos #URL# Unete #USER#]]></document>\n\t\t<document><![CDATA[VIDEO \xe2\x80\x93 Cineasta \xe2\x80\x9c

## Models definition

In [ ]:
# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)

# English Model Training.
model_en = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)

# Spanish Model Training. 
model_es = ClassificationModel("roberta", 
                                  'PlanTL-GOB-ES/roberta-base-bne', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)

num_epochs_per_run = 10
num_runs = 5

Cuda available?  True


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

## Training and evaluation of English model

In [ ]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
  epochs_accuracy=[]
  model_en = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
  for epoch in range (0,num_epochs_per_run):
    # train model
    model_en.train_model(train_df_en)
    result_en, model_outputs_en, wrong_predictions_en = model_en.eval_model(test_df_en)
    # Results on english dataset.
    print(result_en)
    correct_predictions = result_en['tp']+result_en['tn']
    print("Correct predictions are: ",correct_predictions)
    total_predictions = result_en['tp']+result_en['tn']+result_en['fp']+result_en['fn']
    print("Total predictions are: ",total_predictions)
    accuracy = correct_predictions/total_predictions
    print("Accuracy on English test set is:",accuracy,"\n\n")
    epochs_accuracy.append(accuracy)

  print(epochs_accuracy)
  runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on English are:", runs_accuracy)
print("The median for English is:",runs_accuracy[2])


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/38 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/38 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

{'mcc': 0.27615793931820587, 'tp': 53, 'tn': 74, 'fp': 26, 'fn': 47, 'auroc': 0.66345, 'auprc': 0.6353456975033015, 'eval_loss': 1.285394458770752}
Correct predictions are:  127
Total predictions are:  200
Accuracy on English test set is: 0.635 


[0.675, 0.53, 0.575, 0.635, 0.675, 0.695, 0.685, 0.64, 0.675, 0.635]


 Over all runs maximum accuracies on English are: [0.68, 0.695, 0.705, 0.715, 0.74]
The median for English is: 0.705


## Training and evaluation of Spanish model

In [ ]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
  epochs_accuracy=[]
  model_es = ClassificationModel("roberta", 
                                  'PlanTL-GOB-ES/roberta-base-bne', 
                                  args = model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
  for epoch in range (0,num_epochs_per_run):
    # train model
    model_es.train_model(train_df_es)
    result_es, model_outputs_es, wrong_predictions_es = model_es.eval_model(test_df_es)
    # Results on english dataset.
    print(result_es)
    correct_predictions = result_es['tp']+result_es['tn']
    print("Correct predictions are: ",correct_predictions)
    total_predictions = result_es['tp']+result_es['tn']+result_es['fp']+result_es['fn']
    print("Total predictions are: ",total_predictions)
    accuracy = correct_predictions/total_predictions
    print("Accuracy on Spanish test set is:",accuracy,"\n\n")
    epochs_accuracy.append(accuracy)

  print(epochs_accuracy)
  runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on Spanish are:", runs_accuracy)
print("The median for Spanish is:",runs_accuracy[2])


Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/38 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/38 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

{'mcc': 0.31001550116259685, 'tp': 65, 'tn': 66, 'fp': 34, 'fn': 35, 'auroc': 0.7278000000000001, 'auprc': 0.7282690318806229, 'eval_loss': 3.2119966745376587}
Correct predictions are:  131
Total predictions are:  200
Accuracy on Spanish test set is: 0.655 


[0.695, 0.68, 0.73, 0.68, 0.695, 0.72, 0.685, 0.675, 0.675, 0.655]


 Over all runs maximum accuracies on Spanish are: [0.725, 0.73, 0.735, 0.74, 0.74]
The median for Spanish is: 0.735
